## 텍스트 정규화

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.datasets import fetch_20newsgroups

news_data = fetch_20newsgroups(subset='all', random_state=156)

In [3]:
# key
print(news_data.keys())

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])


20개의 카테고리

In [4]:
import pandas as pd

print('target 클래스 값과 분포도', pd.Series(news_data.target).value_counts().sort_index(), sep='\n')
print('target 클래스 이름', news_data.target_names, sep='\n')

target 클래스 값과 분포도
0     799
1     973
2     985
3     982
4     963
5     988
6     975
7     990
8     996
9     994
10    999
11    991
12    984
13    990
14    987
15    997
16    910
17    940
18    775
19    628
dtype: int64
target 클래스 이름
['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']


뉴스그룹 제목, 작성자, 소속, 이메일 등이 있으며 header와 footer를 제거해야함

In [5]:
print(news_data.data[0])

From: egreen@east.sun.com (Ed Green - Pixel Cruncher)
Subject: Re: Observation re: helmets
Organization: Sun Microsystems, RTP, NC
Lines: 21
Distribution: world
Reply-To: egreen@east.sun.com
NNTP-Posting-Host: laser.east.sun.com

In article 211353@mavenry.altcit.eskimo.com, maven@mavenry.altcit.eskimo.com (Norman Hamer) writes:
> 
> The question for the day is re: passenger helmets, if you don't know for 
>certain who's gonna ride with you (like say you meet them at a .... church 
>meeting, yeah, that's the ticket)... What are some guidelines? Should I just 
>pick up another shoei in my size to have a backup helmet (XL), or should I 
>maybe get an inexpensive one of a smaller size to accomodate my likely 
>passenger? 

If your primary concern is protecting the passenger in the event of a
crash, have him or her fitted for a helmet that is their size.  If your
primary concern is complying with stupid helmet laws, carry a real big
spare (you can put a big or small head in a big helmet, bu

In [6]:
%%time

from sklearn.datasets import fetch_20newsgroups

# 학습 데이터의 내용만 추출
train_news = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'),
                  random_state=156)
X_train = train_news.data
y_train = train_news.target

# 테스트 데이터의 내용만 추출
test_news = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'),
                  random_state=156)
X_test = test_news.data
y_test = test_news.target

print(f'학습 데이터 크기 {len(X_train)}, 테스트 데이터 크기 {len(X_test)}')

학습 데이터 크기 11314, 테스트 데이터 크기 7532
Wall time: 1.86 s


## 피처 벡터화 변환 1 - `CountVectorizer`
- `CountVectorizer` 적용할 때 반드시 학습 데이터를 이용해 fit()이 수행된 객체를 이용해 테스트 데이터를 transform() 해야함
    - 그렇지 않으면 피처 개수가 달라져서 학습되지 않음
    - 그렇기 때문에 테스트 데이터에 fit_transform()을 사용하면 안 됨

In [7]:
%%time

from sklearn.feature_extraction.text import CountVectorizer

cnt_vect = CountVectorizer()
cnt_vect.fit(X_train)
X_train_cnt_vect = cnt_vect.transform(X_train)

X_test_cnt_vect = cnt_vect.transform(X_test)

print('학습 데이터 텍스트의 CountVectorizer Shape :', X_train_cnt_vect.shape)
print('테스트 데이터 텍스트의 CountVectorizer Shape :', X_test_cnt_vect.shape)

학습 데이터 텍스트의 CountVectorizer Shape : (11314, 101631)
테스트 데이터 텍스트의 CountVectorizer Shape : (7532, 101631)
Wall time: 3.58 s


### 머신러닝 모델 학습/예측/평가

In [8]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lr_clf = LogisticRegression()
lr_clf.fit(X_train_cnt_vect, y_train)
pred = lr_clf.predict(X_test_cnt_vect)
acc = accuracy_score(y_test, pred)

print(f'CountVectorized Logistic Regression 예측 정확도 : {acc:.3f}')

CountVectorized Logistic Regression 예측 정확도 : 0.608
Wall time: 29.3 s


## 피처 벡터화 변환 2 - `TF-IDF`

In [9]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

print('학습 데이터 텍스트의 TFIDFVectorizer Shape :', X_train_tfidf_vect.shape)
print('테스트 데이터 텍스트의 TFIDFVectorizer Shape :', X_test_tfidf_vect.shape)

학습 데이터 텍스트의 TFIDFVectorizer Shape : (11314, 101631)
테스트 데이터 텍스트의 TFIDFVectorizer Shape : (7532, 101631)
Wall time: 3.52 s


### 머신러닝 모델 학습/예측/평가

In [10]:
%%time

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
acc = accuracy_score(y_test, pred)

print(f'TF-IDF Logistic Regression 예측 정확도 : {acc:.3f}')

TF-IDF Logistic Regression 예측 정확도 : 0.674
Wall time: 25.6 s


## 피처 벡터화 변환 3 - 피처 전처리
- stop word 제거
- ngram_range 확장
- max_df을 설정하여 많이 등장하는 단어를 피처에서 제외

In [11]:
%%time

tfidf_vect = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_df=300)
tfidf_vect.fit(X_train)
X_train_tfidf_vect = tfidf_vect.transform(X_train)
X_test_tfidf_vect = tfidf_vect.transform(X_test)

print('학습 데이터 텍스트의 TF-IDFVectorizer Shape :', X_train_tfidf_vect.shape)
print('테스트 데이터 텍스트의 TF-IDFVectorizer Shape :', X_test_tfidf_vect.shape)

학습 데이터 텍스트의 TF-IDFVectorizer Shape : (11314, 943453)
테스트 데이터 텍스트의 TF-IDFVectorizer Shape : (7532, 943453)
Wall time: 8.18 s


### 머신러닝 모델 학습/예측/평가

In [12]:
%%time

lr_clf = LogisticRegression()
lr_clf.fit(X_train_tfidf_vect, y_train)
pred = lr_clf.predict(X_test_tfidf_vect)
acc = accuracy_score(y_test, pred)

print(f'TF-IDF Vectorized Logistic Regression 예측 정확도 : {acc:.3f}')

TF-IDF Vectorized Logistic Regression 예측 정확도 : 0.692
Wall time: 2min 55s


### 머신러닝 모델 최적 파라미터 조정

In [13]:
%%time

from sklearn.model_selection import GridSearchCV

# CV fold = 3으로 그리드 서치 진행
params = {'C':[0.01, 0.1, 1, 5, 10]}
grid_cv_lr = GridSearchCV(lr_clf, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_lr.fit(X_train_tfidf_vect, y_train)
print('Logistic Regression best C parameter :', grid_cv_lr.best_params_)

# 최적 파라미터로 예측 및 평가
pred = grid_cv_lr.predict(X_test_tfidf_vect)
acc = accuracy_score(y_test, pred)

print(f'TF-IDF Vectorized Logistic Regression 예측 정확도 : {acc:.3f}')

Fitting 3 folds for each of 5 candidates, totalling 15 fits
Logistic Regression best C parameter : {'C': 10}
TF-IDF Vectorized Logistic Regression 예측 정확도 : 0.701
Wall time: 41min 5s


## 파이프라인 사용
- 대용량 데이터의 피처 벡터화 결과를 별도 데이터로 저장하지 않고 바로 머신러닝 알고리즘의 데이터로 입력할 수 있기 때문에 시간 절약

In [15]:
%%time

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# TfidVectorizer의 fit, transform, LogisticRegression의 fit, predict의 과정이 필요 없으며
# 변수 저장도 하지 않음
pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'Pipeline을 통한 Logistic Regression의 예측 정확도 : {acc:.3f}')

Pipeline을 통한 Logistic Regression의 예측 정확도 : 0.701
Wall time: 4min 12s


### GridSearchCV와 결합
- 객체별로 파라미터가 다르기 때문에 객체명__파라미터명으로 결합된 것을 파라미터로 준다.
- 단점은 객체별 파라미터를 주기 때문에 수행시간이 오래 걸린다.
- ```python
{'tfidf_vect__max_df':[100, 300, 700]} 등
```

In [17]:
%%time

from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('lr_clf', LogisticRegression(C=10))
])

# 파라미터 정의
params = {'tfidf_vect__ngram_range':[(1, 1), (1, 2), (1, 3)],
         'tfidf_vect__max_df':[100, 300, 700],
         'lr_clf__C':[1, 5, 10]}

# GridSearchCV에 estimator 대신 pipeline 입력
grid_cv_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_cv_pipe.fit(X_train, y_train)
print(grid_cv_pipe.best_params_, grid_cv_pipe.best_score_)

pred = grid_cv_pipe.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'Pipeline을 통한 Logistic Regression의 예측 정확도 : {acc:.3f}')

Fitting 3 folds for each of 27 candidates, totalling 81 fits
{'lr_clf__C': 10, 'tfidf_vect__max_df': 300, 'tfidf_vect__ngram_range': (1, 2)} 0.7536687914006531
Pipeline을 통한 Logistic Regression의 예측 정확도 : 0.701
Wall time: 3h 43min 31s


## 추가 알고리즘
- 희소 행렬에 잘 사용되는 알고리즘으로 SVM, Naive Bayes가 있다.

### Support Vector Machine

In [18]:
from sklearn.svm import SVC

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('svm_clf', SVC())
])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'Pipeline을 통한 SVC의 예측 정확도 : {acc:.3f}')

Pipeline을 통한 SVC의 예측 정확도 : 0.678


### Naive Bayes

In [19]:
%%time

from sklearn.naive_bayes import MultinomialNB

pipeline = Pipeline([
    ('tfidf_vect', TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=300)),
    ('nb_clf', MultinomialNB())
])

pipeline.fit(X_train, y_train)
pred = pipeline.predict(X_test)
acc = accuracy_score(y_test, pred)
print(f'Pipeline을 통한 Naive Bayes의 예측 정확도 : {acc:.3f}')

Pipeline을 통한 Naive Bayes의 예측 정확도 : 0.689
Wall time: 6.46 s
